In [120]:
import pandas as pd
import numpy as np
from scipy.spatial import Delaunay
from collections import Counter
import os
from datetime import datetime
import re

In [6]:
def count_interactions(data, triangulation_data, colname):
    tri = triangulation_data
    labels = list(data[colname])
    labels_df = pd.DataFrame({'label': labels})
    pairs = []
    for simplex in tri.simplices:
        # Get the labels of the triangle’s three points
        simplex_labels = labels_df.loc[simplex, 'label'].values
        
        for i in range(3):
            for j in range(i + 1, 3):
                pair = tuple(sorted([simplex_labels[i], simplex_labels[j]]))
                pairs.append(pair)
    
    # Count unordered pairs
    pair_counts = Counter(pair for pair in pairs)
    # pair_counts = Counter(pair for pair in pairs_test)
    
    # Build list of rows for DataFrame
    counts_data = [("{}".format('-'.join(sorted(pair))), count) for pair, count in pair_counts.items()]
    
    # Create DataFrame
    interactions_count_df = pd.DataFrame(counts_data, columns=['interaction', 'interaction_count'])
    
    return interactions_count_df


def count_interactions_delaunay(data, colname):
    points = data[['x', 'y']].to_numpy()
    tri = Delaunay(points)
    interactions_count_df = count_interactions(data, tri, colname)

    return interactions_count_df

In [52]:
def create_interactions_count_df(files_list, outdir='data', write=True):

    interactions_list = []
    
    for file in files_list:
        match = re.search(r'/([^/]+)_analysis/', file)
        patient_id = match.group(1) if match else None
    
        print(f"Loading {file}")
        data = pd.read_csv(file, index_col=0)
    
        print("Counting interactions...")
        interactions_count_df = count_interactions_delaunay(data, "phenotype")
        interactions_count_df["patient_id"] = patient_id
    
        print(interactions_count_df.head())
    
        # Get current datetime
        now = datetime.now()
        
        os.makedirs('tmp', exist_ok=True)
    
        output_file = os.path.join('tmp', f"{patient_id}_interactions_{now.strftime('%Y%m%d%H%M')}.csv")
    
        print(f"Saving to {output_file}")
        interactions_count_df.to_csv(output_file)
    
        print("Adding to list...")
        interactions_list.append(interactions_count_df)

    interactions_df = pd.concat(interactions_list, ignore_index=True)

    if write:
        interactions_df.to_csv(os.path.join(outdir, f"cell_type_interactions_{now.strftime('%Y%m%d%H%M')}.csv"), index=False)

    return interactions_df

In [113]:
filename = 'phenotypes.csv'
filename = 'phenotypes_20250723.csv'
root_dir = '/hpcnfs/techunits/imaging/work/ATTEND/analysis_runs'

patient_ids_list = [
    '23S64',
    '35138',
    '19E110099A4',
    '1826315',
    'B19-10215',
    '21S188',
    '21-37601',
    '1923575',
    '23S22',
    '1818600'
    ]



files_list = [os.path.join(root_dir, f"{patient_id}_analysis", "output", filename) for patient_id in patient_ids_list]

In [54]:
interactions_df = create_interactions_count_df(files_list, outdir='data', write=True)

Loading /hpcnfs/techunits/imaging/work/ATTEND/analysis_runs/23S64_analysis/output/phenotypes_20250723.csv
Counting interactions...
                 interaction  interaction_count  patient_id
0      Stroma SMA-Stroma SMA             544506  phenotypes
1    Stroma SMA-Unclassified             431332  phenotypes
2  Unclassified-Unclassified             776817  phenotypes
3                Tumor-Tumor             461062  phenotypes
4         Tumor-Unclassified              51786  phenotypes
Saving to tmp/phenotypes_interactions_202507291242.csv
Adding to list...
Loading /hpcnfs/techunits/imaging/work/ATTEND/analysis_runs/35138_analysis/output/phenotypes_20250723.csv
Counting interactions...
                 interaction  interaction_count  patient_id
0  Unclassified-Unclassified             824212  phenotypes
1         PAX2+-Unclassified               2832  phenotypes
2                PAX2+-PAX2+                372  phenotypes
3    Stroma SMA-Unclassified             658154  phenotypes
4    

In [65]:
interactions_df.head()

,interaction,interaction_count,patient_id
0,Stroma SMA-Stroma SMA,544506,phenotypes
1,Stroma SMA-Unclassified,431332,phenotypes
2,Unclassified-Unclassified,776817,phenotypes
3,Tumor-Tumor,461062,phenotypes
4,Tumor-Unclassified,51786,phenotypes


In [101]:
# interactions_df.to_csv('data/phenotypes_interactions_20250730.csv', index=False)